# GOALS

* Decide on viewcount threshold to eliminate views?
* Get feature columns working
* Build an LDA model

In [1]:
## testing printing output from console
import subprocess
cmd = [ 'echo', '"Welcome to my PySpark analysis of some StackExchange Data"' ]
output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
print(output)

b'"Welcome to my PySpark analysis of some StackExchange Data"\n'


In [2]:
import time
import numpy as np
import pandas as pd
from datetime import datetime

# Load PySpark and Data Structures

In [3]:
%run -i '1-load-pyspark-and-structs.py'

The Spark UI is available at: http://192.168.0.26:4040/ and the defaultParallelism is 4
The data_array is: ['english', 'math', 'rus_stackoverflow', 'stackoverflow', 'superuser']


# Load Data

In [4]:
%%time
print(datetime.now().time())
#%run -i '2-load-datasets.py'

14:07:11.798877
CPU times: user 108 µs, sys: 83 µs, total: 191 µs
Wall time: 130 µs


In [5]:
def show_spark_df(df, n=5):
    '''
    function to better print spark df entries
    '''
    display(pd.DataFrame(df.head(n), columns=df.columns))

# Clean Data

In [6]:
%%time
print(datetime.now().time())
#%run -i '3-clean-datasets.py'

14:07:11.828323
CPU times: user 206 µs, sys: 71 µs, total: 277 µs
Wall time: 213 µs


# Define Target, EDA and Export

In [7]:
%%time
print(datetime.now().time())
#%run -i '4-define-target-eda-export.py'

#NB TO DO: Find threshold to delete low views to make sure users that can vote have seen the question

'''
vc_thresh_data = {}

## finding means of viewcounts across fora

for i in data_array:
    vc_thresh_data[i] = datasets[i].select("viewcount").rdd.flatMap(lambda x: x).mean()

vc_thresh_data'''

14:07:11.838585
CPU times: user 183 µs, sys: 72 µs, total: 255 µs
Wall time: 201 µs


# Load Clean Data

In [8]:
%%time

## load clean data
for i in data_array:
    datasets[i] = (
        spark
        .read
        .load(f'clean-data/{i}.parquet')
)

CPU times: user 3.92 ms, sys: 2.35 ms, total: 6.27 ms
Wall time: 3.56 s


# Train/Test Split

In [9]:
%%time
        
## create train and test dictionaries
train = {}
test = {}

from pyspark.sql.functions import lit

## assign train and test sets, based on after certain date
for i in data_array:
    date = '2018-01-01' # date used to split data
    train[i] = datasets[i].filter(datasets[i]['clean_date'] < lit(date))
    test[i] = datasets[i].filter(datasets[i]['clean_date'] >= lit(date))
    numer = test[i].count()
    denom = train[i].count()
    frac = numer/denom
    print(f'{i}: {frac}, from {numer} and {denom}')

english: 0.20050307879670837, from 17616 and 87859
math: 0.22927435077324415, from 198276 and 864798
rus_stackoverflow: 0.4540735776836319, from 79956 and 176086
stackoverflow: 0.20690967926144474, from 60872 and 294196
superuser: 0.12467468017854172, from 44216 and 354651
CPU times: user 24.3 ms, sys: 7.43 ms, total: 31.7 ms
Wall time: 13 s


In [10]:
'''
interesting to see how skewed rus_stackoverflow posts are to more posts in recent years
'''

'\ninteresting to see how skewed rus_stackoverflow posts are to more posts in recent years\n'

# Create Results Dictionary

In [11]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

# Silly Mean and Median Models

In [12]:
from pyspark.sql.functions import array, lit, struct

## create mean dictionaries
y_ravi_means = {}

## calculate the mean of each forum, using ONLY train set
for i in data_array:
    y_ravi_means[i] = train[i].select('y_ravi').rdd.flatMap(lambda x: x).mean()

In [13]:
## import rmse evaluator
from pyspark.ml.evaluation import RegressionEvaluator

## create baselines dictionary
base = {}

## modelling
for i in data_array:

    ## initial variable for timing
    t0 = time.time()
    
    ## train silly mean model by assigning full mean to each row of test
    test[i] = test[i].withColumn('mean_pred', lit(y_ravi_means[i]))

    ## evaluate silly mean model, ONLY on test set
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="y_ravi", predictionCol="mean_pred")
    base[i] = round( evaluator.evaluate(test[i]), 6)

    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m mean\033[0m model is {base[i]}")

    ## record time taken
    timt = round( time.time() - t0, 2 )
    
    ## store as dictionary inside RESULTS dictionary, initiating dataset name entries first
    RESULTS[i.title()]['0silly_mean.rmse'] = base[i]
    RESULTS[i.title()]['0silly_mean.timet'] = timt

The root-mean-square error of english's mean model is 0.020986
The root-mean-square error of math's mean model is 0.027263
The root-mean-square error of rus_stackoverflow's mean model is 0.023957
The root-mean-square error of stackoverflow's mean model is 0.02202
The root-mean-square error of superuser's mean model is 0.02028


In [14]:
def show_save_results(results, filename='final-results.csv'):
    '''
    function to print and export modelling results
    '''
    display(pd.DataFrame.from_dict(results).T)
    print(pd.DataFrame.from_dict(results).T.to_latex())
    pd.DataFrame.from_dict(results).T.to_csv(filename)

In [15]:
show_save_results(RESULTS)

,silly_mean.rmse,silly_mean.timet
English,0.020986,1.23
Math,0.027263,1.97
Rus_Stackoverflow,0.023957,0.83
Stackoverflow,0.022020,0.85
Superuser,0.020280,0.87


\begin{tabular}{lrr}
\toprule
{} &  silly\_mean.rmse &  silly\_mean.timet \\
\midrule
English           &         0.020986 &              1.23 \\
Math              &         0.027263 &              1.97 \\
Rus\_Stackoverflow &         0.023957 &              0.83 \\
Stackoverflow     &         0.022020 &              0.85 \\
Superuser         &         0.020280 &              0.87 \\
\bottomrule
\end{tabular}



# Viewcount Model

In [16]:
%%time
#3min 56s

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'y_ravi'
numic_variables = ['viewcount']
datet_variables = ['clean_date']

## numerical columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression on just viewcount
from pyspark.ml.regression import LinearRegression
    
lr = LinearRegression(#maxIter=100, # this doesn't change anything
                      #regParam=0.3, # using regularisation parameter here useless since there is one feature
                      #elasticNetParam=0.8,
                      featuresCol="features",
                      labelCol=target,
                      predictionCol="viewcount_pred")

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
#.addGrid(lr.regParam, [1e-3, 1.])
#.addGrid(lr.elasticNetParam, [1e-3, 1.])
paramGrid = ParamGridBuilder() \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="y_ravi", predictionCol="viewcount_pred")

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## this fits on train, then predicts on test, using 3-CV
    rmse = round( evaluator.evaluate(crossval.fit(train[i]).transform(test[i])), 6)
    
    ''' CODE TO USE LATER
    cvModel = final_pipeline.fit(datasets[i])
    cvModel.avgMetrics
    list(zip(cvModel.avgMetrics, paramGrid))
    cvModel.bestModel.stages
    cvModel.bestModel.stages[1].extractParamMap()
    '''
        
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m viewcount\033[0m model is {rmse}")

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 3 )
    
    ## record time taken
    timt = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['1viewcount.rmse'] = rmse
    RESULTS[i.title()]['1viewcount.v.imprv'] = impr
    RESULTS[i.title()]['1viewcount.timet'] = timt
    
## record results
show_save_results(RESULTS, 'test-set-results.csv')

The root-mean-square error of english's viewcount model is 0.020949
The root-mean-square error of math's viewcount model is 0.027226
The root-mean-square error of rus_stackoverflow's viewcount model is 0.023914
The root-mean-square error of stackoverflow's viewcount model is 0.022019
The root-mean-square error of superuser's viewcount model is 0.020267


,silly_mean.rmse,silly_mean.timet,viewcount.rmse,viewcount.timet,viewcount.v.imprv
English,0.020986,1.23,0.020949,17.27,0.176
Math,0.027263,1.97,0.027226,62.91,0.136
Rus_Stackoverflow,0.023957,0.83,0.023914,21.44,0.179
Stackoverflow,0.022020,0.85,0.022019,31.06,0.005
Superuser,0.020280,0.87,0.020267,24.81,0.064


\begin{tabular}{lrrrrr}
\toprule
{} &  silly\_mean.rmse &  silly\_mean.timet &  viewcount.rmse &  viewcount.timet &  viewcount.v.imprv \\
\midrule
English           &         0.020986 &              1.23 &        0.020949 &            17.27 &              0.176 \\
Math              &         0.027263 &              1.97 &        0.027226 &            62.91 &              0.136 \\
Rus\_Stackoverflow &         0.023957 &              0.83 &        0.023914 &            21.44 &              0.179 \\
Stackoverflow     &         0.022020 &              0.85 &        0.022019 &            31.06 &              0.005 \\
Superuser         &         0.020280 &              0.87 &        0.020267 &            24.81 &              0.064 \\
\bottomrule
\end{tabular}

CPU times: user 1.26 s, sys: 379 ms, total: 1.64 s
Wall time: 2min 37s


In [17]:
'''
Interesting that there are different improvements of viewcount over mean-only prediciton
'''

'\nInteresting that there are different improvements of viewcount over mean-only prediciton\n'

# Begin Feature Engineering

In [18]:
## garbage collector to speed up computation
import gc
collected = gc.collect()
print('Garbage collector: collected %d objects.' % collected)

Garbage collector: collected 1399 objects.


In [ ]:
import nltk

from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

## custom transformer for nltk tokenisation

class NLTKWordPunctTokenizer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, stopwords=None):
        super(NLTKWordPunctTokenizer, self).__init__()
        self.stopwords = Param(self, "stopwords", "")
        self._setDefault(stopwords=set())
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, stopwords=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def setStopwords(self, value):
        self._paramMap[self.stopwords] = value
        return self

    def getStopwords(self):
        return self.getOrDefault(self.stopwords)

    def _transform(self, dataset):
        stopwords = self.getStopwords()

        def f(s):
            tokens = nltk.tokenize.wordpunct_tokenize(s)
            return [t for t in tokens if t.lower() not in stopwords]

        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [48]:
%%time
# 13min

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'y_ravi'
textt_variables = ['title', 'clean_body']
datet_variables = ['clean_date']

## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')

## textual columns
# tokenising text cols with custom transformer
nltk_tokenizer_body = NLTKWordPunctTokenizer(
    inputCol='clean_body', outputCol='body_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

nltk_tokenizer_title = NLTKWordPunctTokenizer(
    inputCol='title', outputCol='titl_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

# count occurence of tokens
cnt_vectrizr_body = CountVectorizer(inputCol='body_words', outputCol='body_feats', minDF=2)
cnt_vectrizr_title = CountVectorizer(inputCol='titl_words', outputCol='titl_feats', minDF=2)



## create processing pipeline
process_assembler = VectorAssembler(inputCols=['body_feats', 'titl_feats'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[  #inputCols=['datet_data']
    nltk_tokenizer_body, 
    nltk_tokenizer_title,
    cnt_vectrizr_body,
    cnt_vectrizr_title,
    process_assembler
])

########################
##### CHOOSE MODEL #####
########################

## linear regression on just viewcount
from pyspark.ml.regression import LinearRegression
    
lr = LinearRegression(#maxIter=100, # this doesn't change anything
                      #regParam=0.3, # using regularisation parameter here useless since there is one feature
                      #elasticNetParam=0.8,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='tokens_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [1e-3, 1.]) \
    .addGrid(lr.elasticNetParam, [1e-3, 1.]) \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol='y_ravi', predictionCol='tokens_pred')

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

## modelling JUST USE ENGLISH FOR NOW
for i in ['english']:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## this fits on train, then predicts on test using 3-CV
    rmse = round( evaluator.evaluate(crossval.fit(train[i]).transform(test[i])), 6)
    
    ''' CODE TO USE LATER
    cvModel = final_pipeline.fit(datasets[i])
    cvModel.avgMetrics
    list(zip(cvModel.avgMetrics, paramGrid))
    cvModel.bestModel.stages
    cvModel.bestModel.stages[1].extractParamMap()
    '''
        
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m tokens\033[0m model is {rmse}")

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 3 )
    
    ## record time taken
    timt = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2tokens.rmse'] = rmse
    RESULTS[i.title()]['2tokens.v.imprv'] = impr
    RESULTS[i.title()]['2tokens.timet'] = timt
    
## record results
show_save_results(RESULTS, 'test-set-results.csv')

The root-mean-square error of english's viewcount model is 0.032631


KeyboardInterrupt: 

In [50]:
show_save_results(RESULTS, 'test-set-results.csv')

,silly_mean.rmse,silly_mean.timet,tokens.rmse,tokens.timet,tokens.v.imprv,viewcount.rmse,viewcount.timet,viewcount.v.imprv
English,0.020986,1.23,0.032631,800.51,-55.489,0.020949,17.27,0.176
Math,0.027263,1.97,NaN,NaN,NaN,0.027226,62.91,0.136
Rus_Stackoverflow,0.023957,0.83,NaN,NaN,NaN,0.023914,21.44,0.179
Stackoverflow,0.022020,0.85,NaN,NaN,NaN,0.022019,31.06,0.005
Superuser,0.020280,0.87,NaN,NaN,NaN,0.020267,24.81,0.064


\begin{tabular}{lrrrrrrrr}
\toprule
{} &  silly\_mean.rmse &  silly\_mean.timet &  tokens.rmse &  tokens.timet &  tokens.v.imprv &  viewcount.rmse &  viewcount.timet &  viewcount.v.imprv \\
\midrule
English           &         0.020986 &              1.23 &     0.032631 &        800.51 &         -55.489 &        0.020949 &            17.27 &              0.176 \\
Math              &         0.027263 &              1.97 &          NaN &           NaN &             NaN &        0.027226 &            62.91 &              0.136 \\
Rus\_Stackoverflow &         0.023957 &              0.83 &          NaN &           NaN &             NaN &        0.023914 &            21.44 &              0.179 \\
Stackoverflow     &         0.022020 &              0.85 &          NaN &           NaN &             NaN &        0.022019 &            31.06 &              0.005 \\
Superuser         &         0.020280 &              0.87 &          NaN &           NaN &             NaN &        0.020267 &       

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 57194)
Traceback (most recent call last):
  File "/Users/brad/anaconda3/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/brad/anaconda3/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/brad/anaconda3/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/brad/anaconda3/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/accumulators.py", l

In [20]:
## import elements from natural language toolkit
import nltk
#nltk.download('all') # uncomment after first run as admin check
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop_words = set(stopwords.words('english'))
table = str.maketrans('', '', string.punctuation)
lmtzr = WordNetLemmatizer()

def get_tokens(line):
    '''
    Function to parse text features
    '''
    tokens = word_tokenize(line)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuations from each word
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stopwords
    words = [w for w in words if not w in stop_words]
    # lemmatizing the words, see https://en.wikipedia.org/wiki/Lemmatisation
    '''lemmatise or stem???'''
    words = [lmtzr.lemmatize(w) for w in words]
    # remove single letters
    words = [word for word in words if not len(word)==1]
    return (words)

In [22]:
import pyspark.sql.functions as F
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import Bucketizer
from pyspark.sql import DataFrame
from typing import Iterable
import pandas as pd

## custom transformer to spread sparse vectors into individual columns
class VectorMLliber(Transformer):
    """
    A custom Transformer which converts a column of pyspark.ml vectors to multiple pyspark.mllib vectors.
    """

    def __init__(self, inputCol=None):
        super(VectorMLliber, self).__init__()

    def _transform(self, df: DataFrame) -> DataFrame:
        
        def f(v):
            return Vectors.sparse(v.size, v.indices, v.values)
        
        df = df.rdd.map(lambda r: as_mllib_vector(r[0]))
        return df

In [ ]:
# ???????????
VectorMLliber_body = VectorMLliber(inputCol='body_features')
VectorMLliber_title = VectorMLliber(inputCol='titl_features')

In [23]:
'''def as_mllib_vector(v):
    return Vectors.sparse(v.size, v.indices, v.values)

features = {}
feature_vec_list = {}
for i in data_array:
    features[i] = word_feat_list[i].select("features")
    feature_vec_list[i] = features[i].rdd.map(lambda r: as_mllib_vector(r[0]))
    feature_vec_list[i].cache()
'''

'def as_mllib_vector(v):\n    return Vectors.sparse(v.size, v.indices, v.values)\n\nfeatures = {}\nfeature_vec_list = {}\nfor i in data_array:\n    features[i] = word_feat_list[i].select("features")\n    feature_vec_list[i] = features[i].rdd.map(lambda r: as_mllib_vector(r[0]))\n    feature_vec_list[i].cache()\n'

In [29]:
(trained_pipeline
 .transform(datasets['english'])
 .select(
    indep_text_variables + ["prediction"]
 )
 .write
 .parquet("linreg_prediction.parquet")
)

AnalysisException: 'path file:/Users/brad/Dropbox/lse-msc-thesis-brad/01-python-code/00-workspace/linreg_prediction.parquet already exists.;'

In [ ]:
linreg_predictions = spark.read.parquet("linreg_prediction.parquet")

In [ ]:
linreg_predictions.toPandas().head()

In [ ]:
linreg_predictions.select("prediction").describe().toPandas()

# Save pipelines

In [ ]:
from joblib import dump, load
dump(estimator_pipeline, 'pipeline.joblib') 

reloaded = load("pipeline.joblib")

Now we can predict directly!

reloaded.predict(X)[:10]

# Convert notebook to python file

In [ ]:
!jupyter nbconvert --to script 0-master-notebook-pipelines.ipynb